In [ ]:
import cv2
import numpy as np
from tensorflow import keras
from keras.optimizers import Adam
from keras.utils import np_utils
#from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D

In [ ]:
CLASS_MAP = {
    "paper": 0,
    "rock": 1,
    "scissor": 2,
}



def mapper(val):
    return CLASS_MAP[val]

In [ ]:
IMG_SAVE_PATH='image_data'
os.listdir(IMG_SAVE_PATH)

In [ ]:
# creating dataset from training data images and their labels (labels=their folder name)
dataset = []

for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (112, 112))
        dataset.append([img, directory])

In [ ]:
type(dataset)

In [ ]:
import random
random.shuffle(dataset)

In [ ]:
data, labels = zip(*dataset)

In [ ]:
X_train=np.array(data)

In [ ]:
X_train=X_train.astype(float)

In [ ]:
X_train=X_train/255

In [ ]:
labels = list(map(mapper, labels))

In [ ]:
y_train=np.array(labels)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#splitting data 
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [ ]:
def get_model():
    cnn = models.Sequential([
        
    # Convolution Layer Conv1
        
    layers.Conv2D(filters=64, kernel_size=(3, 3),input_shape=(112, 112, 3),activation ='relu'),
    layers.MaxPooling2D((2, 2)),
    
    #Convolution layer Conv2
        
    layers.Conv2D(filters=128, kernel_size=(3, 3),activation ='relu'),
    layers.MaxPooling2D((2, 2)),
        
    #3
        
    layers.Conv2D(filters=256, kernel_size=(3, 3),activation ='relu'),
    layers.MaxPooling2D((2, 2)),
        
    #4
        
    layers.Conv2D(filters=512, kernel_size=(3, 3),activation ='relu'),
    layers.MaxPooling2D((2, 2)),
        
    
        
    layers.Flatten(),
    
    layers.Dense(512, activation='relu'),
        
    layers.Dropout(0.5),
        
    layers.Dense(128, activation='relu'),
        
    layers.Dense(4, activation='sigmoid')
])
    return cnn

In [ ]:
model = get_model()

In [ ]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
y_train=y_train.astype(int)
y_test=y_test.astype(int)

In [ ]:
model.fit(X_train,y_train ,epochs=10)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
import numpy as np
y_pred = model.predict(X_test)
y_pred_classes = [np.argmax(element) for element in y_pred]
y_test_classes = [np.argmax(element) for element in y_test]


print("Classification Report: \n", classification_report(y_test_classes, y_pred_classes))

In [ ]:
model.summary()

In [ ]:
model.save("my_model_5")

In [ ]:
def plot_sample(index):
    plt.figure(figsize = (15,2))
    plt.imshow(X_test[index])
    plt.xlabel(y_pred_classes[index])

In [ ]:
plot_sample(27)